Import packages

In [70]:
import numpy as np
import pandas as pd

In [71]:
pd.reset_option('all')
pd.set_option('display.max_columns', None)

Load in required tables from pickle_vars folder

In [72]:
icu_stays = pd.read_pickle('pickle_vars/icustays.pkl.xz')
d_items = pd.read_pickle('pickle_vars/d_items.pkl.xz')
patients = pd.read_pickle('pickle_vars/patients.pkl.xz')
admissions = pd.read_pickle('pickle_vars/admissions.pkl.xz')
chartevents = pd.read_pickle('pickle_vars/chartevents.pkl.xz')

Create list of ITEM_IDs which correspond to measurements of the required variables. The list of IDs is taken directly from TDS.

In [73]:
variable_mapping_dict = {'pain': [223791], 'temp': [223761, 223762, 224027], 'hr': [220045], 'fio2': [227009, 223835], 'resp': [220210, 224688, 224689, 224690], 'airway': [223838, 224832, 224391, 227810, 223837, 224829], 'po2': [226770, 227039, 227516, 220224], 'hco3': [224826, 227443], 'peep': [220339, 224699, 224700], 'gcs': [220739, 223900, 223901, 227011, 227012, 227014, 228112], 'pco2': [220235, 226062, 226063], 'na': [220645, 226534], 'bun': [225624], 'bp': [220050, 220179, 225309, 226850, 226852, 228151], 'creatinine': [220615], 'k': [220640, 227464, 227442, 226535], 'spo2': [220227, 220277, 226860, 226861, 226862, 226863, 226865, 228232], 'haemoglobin': [220228], 'weight': [762, 226512], 'height': [226730, 920, 1394, 4187, 3486, 3485, 4188]}
variable_mapping_list = list(variable_mapping_dict.values())
itemid_list = np.concatenate(variable_mapping_list, dtype = int, casting='unsafe').ravel().tolist()

Query chartevents for rows containing only ITEM_IDs from itemid_list

In [74]:
#mask = chartevents['ITEMID'].isin(itemid_list)
#df = chartevents[mask]
df = chartevents.query('ITEMID.isin(@itemid_list)')

Merge data with icu_stays['INTIME', 'OUTTIME', 'LOS'], and with d_items['LABEL', 'UNITNAME']

In [75]:
df = df.merge(icu_stays[['ICUSTAY_ID','INTIME','OUTTIME','LOS']], on='ICUSTAY_ID')
df = df.merge(d_items[['ITEMID','LABEL','UNITNAME']], on='ITEMID')

Merge with patients table to get DOB and. GENDER, rename GENDER to sex and encode to binary

In [76]:
df = df.merge(patients[['SUBJECT_ID','GENDER','DOB']], on='SUBJECT_ID')
df['sex'] = df.GENDER.replace({'F': 1, 'M': 0})
df = df.drop(columns='GENDER')

Calculate Age

In [77]:
df['age'] = df.apply(lambda s: (s['INTIME'].to_pydatetime()-s['DOB'].to_pydatetime()).days // 365.0, axis=1)

Select only data from MICU, SICU and Metavision

In [78]:
MICU_GICU_META_IDs = icu_stays[((icu_stays.FIRST_CAREUNIT == 'MICU') | (icu_stays.FIRST_CAREUNIT == 'SICU')) & (icu_stays.DBSOURCE == 'metavision')][['ICUSTAY_ID','HADM_ID']]
df = df[(df.ICUSTAY_ID.isin(MICU_GICU_META_IDs.ICUSTAY_ID)) & (df.HADM_ID.isin(MICU_GICU_META_IDs.HADM_ID))]

Add column for hours before discharge: hrs_bd

In [79]:
df['hrs_bd'] = (df['OUTTIME'] - df['CHARTTIME']).dt.total_seconds()/3600

Add column for hours since admission: hrs_SA

In [80]:
df['hrs_SA'] = (df['CHARTTIME'] - df['INTIME']).dt.total_seconds()/3600

Add following columns
* ['outcome']: binary flag. 1 indicates good outcome (survival with no readmission). 0 indicates bad outcome (death or readmission).
* ['in_h_death']: binary flag. 1 for death. 0 for survival.
* ['in_icu_death']: binary flag. 1 for death. 0 for survival.
* ['readmit']: integer. Number of readmissions to icu during same hospital admission.

In [81]:
first_stays_dict = pd.read_pickle('pickle_vars/first_stays.pkl') # HADM_ID:ICUSTAY_ID

mortalities = pd.read_pickle('pickle_vars/mortalities.pkl') # List

icu_deaths = pd.read_pickle('pickle_vars/icu_deaths.pkl') # List

stay_counts = pd.read_pickle('pickle_vars/stay_counts.pkl') # HADM:count

In [82]:
cohort_stays = list(first_stays_dict.values())
cohort_hadms = list(first_stays_dict.keys())

In [83]:
%%time
#df = df.query('ICUSTAY_ID.isin(@cohort_stays) & HADM_ID.isin(@cohort_hadms)')
df['cohort'] = df.apply(lambda row: 1 if (row['ICUSTAY_ID'] in cohort_stays and row['HADM_ID'] in cohort_hadms) else 0, axis=1)

df['in_h_death'] = df['HADM_ID'].apply(lambda hadmid: 1 if hadmid in mortalities else 0)

df['in_icu_death'] = df['ICUSTAY_ID'].apply(lambda icustay: 1 if icustay in icu_deaths else 0 )

df['readmit'] = df['HADM_ID'].apply(lambda hadmid: stay_counts[hadmid] - 1)

df['outcome'] = df['HADM_ID'].apply(lambda hadmid: 1 if (hadmid not in mortalities and stay_counts[hadmid]==1) else 0)

CPU times: user 40min 38s, sys: 6.3 s, total: 40min 44s
Wall time: 40min 44s


In [85]:
print("There are %d unique ICU stays, of which" %df.ICUSTAY_ID.nunique())
print("%d are in the cohort." %df[df['cohort']==1].ICUSTAY_ID.nunique())

There are 14435 unique ICU stays, of which
13607 are in the cohort.


Save df as pkl

In [86]:
df.to_pickle('pickle_vars/chartevents_for_cohort.pkl.xz')